In [27]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import re
import datetime

import pyodbc
from sqlalchemy import create_engine

from faker import Faker
from faker.providers import internet
import random

load_dotenv()

True

In [17]:
fake = Faker(['pt_BR'])
fake.add_provider(internet)

from util.fake_generator import FakeGenerator
fakegen = FakeGenerator(fake)

date_now = datetime.datetime.now()

In [ ]:
file_name = ''
df = pd.read_excel(f'../data/')
df.head(5)

In [ ]:
qtd_linhas = len(df.index)
print(qtd_linhas)
df.columns

In [ ]:
df_purge = pd.DataFrame()

coluna_ref = 'data'
df_purge[coluna_ref] = pd.to_datetime(df[coluna_ref])

# df['ano'] = df[coluna_ref].dt.to_period('Y')
df_purge['ano_mes'] = df_purge[coluna_ref].dt.to_period('M')
distribuicao_mensal = df_purge['ano_mes'].value_counts().sort_index()
distribuicao_mensal

In [ ]:
date_columns = []

for column in date_columns:
    new_column_name = 'delta_' + column
    df_purge[column] = df[column]
    df_purge[new_column_name] = pd.to_datetime(df[column]) - df[coluna_ref]

    media_delta = df_purge[new_column_name].mean()
    max_delta = df_purge[new_column_name].max()
    min_delta = df_purge[new_column_name].min()
    count_empty = df_purge[column].count()
    print(f'### Coluna: {column} ###')
    print(f"Média do tempo delta: {media_delta}")
    print(f"Máximo tempo delta: {max_delta}")
    print(f"Mínimo tempo delta: {min_delta}")
    print(f"Qtd de valores preenchidos: {count_empty} \n")

In [ ]:
df.head(5)

In [83]:
df_faker = df.copy(deep=True)

In [84]:
random.seed(42)

In [85]:
def remove_nan(array):
    return np.array([x for x in array if x is not np.nan])

def generate_date_fake(row, column_name, column_ref):
    if pd.notna(row[column_name]):
        return row[column_name]
    else:
        return random.choice([row[column_ref] + fakegen.gen_random_day_time(0), row[column_ref] + fakegen.gen_random_day_time(0)])


padroes = {
    
    'inventário': re.compile(r'invent[ãaáàâäão]rio+', flags=re.IGNORECASE),
    'provisão': re.compile(r'provis[ãaáàâäão]+', flags=re.IGNORECASE),
    'obsoleto': re.compile(r'obsolet[oa]+', flags=re.IGNORECASE)
}

def encontrar_palavra(texto):
    if pd.notna(texto):
        for palavra, padrao in padroes.items():
            if padrao.search(texto):
                return palavra
    return None

In [ ]:
df_faker['LANCAMENTOS'] = df_faker['Descrição / Justificativa'].apply(encontrar_palavra)
df_faker['LANCAMENTOS'].value_counts()

In [87]:
# TITULO
map_replace = {'': fake.company(), '': fake.company(), 
               '': fake.company(), '': fake.company(), '': fake.company(), '': fake.company()
               }
for key in map_replace:
    new_value = map_replace[key]
    df_faker[''] = df_faker[''].str.replace(key, new_value, regex=False)

# CODIGO
seed_account = df_faker[''].unique()
map_account = {value: f"20{str(fake.unique.random_int(min=1000, max=9999))}" for value in seed_account}
df_faker[''] = df_faker[''].map(map_account)

# FASE ATUAL
df_faker['Fase atual'] = df_faker['Fase atual'].replace({'': '', 
                                                         '': ''})

# CRIADOR 
seed_creator = df_faker['Criador'].unique()
map_creator = {value: fake.ascii_company_email()  for value in seed_creator}
map_creator['Usuário'] = 'Usuário'
del map_creator['Pipebot']
df_faker['Criador'] = df_faker['Criador'].map(map_creator)

# DESCRICAO / JUSTIFICATIVA
seed_description = remove_nan(df_faker['Descrição / Justificativa'].unique())
map_description = {value: fake.paragraph( variable_nb_sentences=True, ext_word_list=['abc', 'def', 'ghi', 'jkl']) for value in seed_description}
df_faker['Descrição / Justificativa'] = df_faker['Descrição / Justificativa'].map(map_description)

# OBSERVAÇÃO
seed_observation = remove_nan(df_faker['Observação'].unique())
map_observation = {value: fake.paragraph( variable_nb_sentences=True, ext_word_list=['abc', 'def', 'ghi', 'jkl']) for value in seed_observation}
df_faker['Observação'] = df_faker['Observação'].map(map_observation)

# Datas Aprovação do Coordenador
df_faker['Primeira vez'] = df_faker.apply(lambda row: generate_date_fake(row, column_name='', 
                                                                                                                         column_ref=''), axis=1)
df_faker['Última vez'] = df_faker.apply(lambda row: generate_date_fake(row, column_name='', 
                                                                                                                     column_ref=''), axis=1)

In [90]:
year_current = date_now.year
month_current = date_now.month

df_faker = df_faker.reset_index(drop=True)
df_faker.to_excel(f'../data/{month_current}.{year_current}.xlsx', index=None)